# DeepFM

在 Wide&Deep 的模型之上用 FM 替换了原来的 Wide 的部分，就成为了现在的 DeepFM

In [ ]:
# build train data

import os
import numpy as np

BASEDIR = os.getcwd()

fields_dict = {}
lines = None

with open(os.path.join(BASEDIR, 'assets/datasets/criteo_ctr/small_train.txt')) as f:
    lines = f.readlines()

fields_dict = {}
for line in lines:
    line = line.strip('\n')

    for elem in line.split(' ')[1:]:
        field, feature, _ = elem.split(':')

        if field not in fields_dict:
            fields_dict[field] = {'index': len(fields_dict), 'features': {}, 'last_idx': -1}

        if feature not in fields_dict[field]['features']:
            fields_dict[field]['features'][feature] = fields_dict[field]['last_idx'] + 1
            fields_dict[field]['last_idx'] = fields_dict[field]['last_idx'] + 1

for field in fields_dict.keys():
    if 'none' not in fields_dict[field]['features']:
        fields_dict[field]['features']['none'] = fields_dict[field]['last_idx'] + 1
        fields_dict[field]['last_idx'] = fields_dict[field]['last_idx'] + 1


def init_field_tensor(fields_dict):
    init_tensor = np.zeros((len(fields_dict), 1))
    for field in fields_dict.keys():
        init_tensor[fields_dict[field]['index']] = fields_dict[field]['last_idx']
    return init_tensor.astype(int)


X_train = []
X_train_sparse = []
y_train = []

for line in lines:
    line = line.strip('\n')
    elems = line.split(' ')
    y_train.append(float(elems[0]))

    init_tensors = init_field_tensor(fields_dict)
    for elem in elems[1:]:
        field, feature, _ = elem.split(':')
        field_idx = fields_dict[field]['index']
        feature_idx = fields_dict[field]['features'][feature]
        init_tensors[field_idx] = feature_idx
    X_train.append(init_tensors)

X_train = np.concatenate(X_train, 1)
y_train = np.array(y_train)

In [ ]:
# build embedding layer

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np


# class CrossBlock(nn.Module):
#     def __init__(self, embedding_size):
#         super(CrossBlock, self).__init__()
        

class FMBlock(nn.Module):
    def __init__(self, fields_dict, embedding_size):
        super(FMBlock, self)
        self.fields_dict = fields_dict
        self.embedding_size = embedding_size

        features = 0
        for field in self.fields_dict.keys():
            features += len(self.fields_dict[field]['features'])

        self.add_block = nn.Linear(features, 1, bias=True).double()

    def forward(self, input_field_tensor, input_sparse_tensor):
        out1 = self.add_block(input_sparse_tensor)
        out2 = self.



class FieldEmbeddingBlock(nn.Module):
    def __init__(self, fields_dict, embedding_size):
        super(FieldEmbeddingBlock, self).__init__()
        self.fields_dict = fields_dict
        self.fields_embedding = {}
        self.embedding_size = embedding_size

        for field in self.fields_dict.keys():
            field_idx = self.fields_dict[field]['index']
            self.fields_embedding[field_idx] = nn.Embedding(len(self.fields_dict[field]['features']),
                                                            self.embedding_size)

    def forward(self, input_field_tensor):
        out = torch.zeros(input_field_tensor.shape[0] * self.embedding_size, input_field_tensor.shape[1])
        idx = 0
        for field in self.fields_dict.keys():
            field_idx = self.fields_dict[field]['index']
            out[idx:idx + self.embedding_size, :] = self.fields_embedding[field_idx](input_field_tensor[field_idx, :])
            idx += self.embedding_size
        return out.double()